In [35]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder
import os

In [36]:
from sklearn.calibration import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


iris = pd.read_csv('Datasets/iris.csv', index_col=0)

#Making trainn and test splits
X, y = iris.drop('Species', axis=1), iris['Species']



In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25, stratify=y)

# le = LabelEncoder()
# iris["Species"] = le.fit_transform(iris['Species'])

scaler = StandardScaler().set_output(transform='pandas')
X_trn_scl = scaler.fit_transform(X_train)

#now applying PCA
prcomp = PCA().set_output(transform='pandas')
X_trn_pca = prcomp.fit_transform(X_trn_scl)

lr = LogisticRegression()
lr.fit(X_trn_pca.iloc[:, :2], y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [38]:
X_tst_scl = scaler.transform(X_test)

#now applying PCA
X_tst_pca = prcomp.transform(X_tst_scl)

y_pred_prob = lr.predict_proba(X_tst_pca.iloc[:, :2])
print(log_loss(y_test, y_pred_prob))
y_pred = lr.predict(X_tst_pca.iloc[:, :2])
print(accuracy_score(y_test, y_pred))

lr = LogisticRegression()
lr.predict(X_tst_pca.iloc[:, :2])

0.16408130072697732
0.9555555555555556


NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
X_scaled = scaler.fit_transform(X)
prcomp = PCA()
X_pca = prcomp.fit_transform(X_scaled)
np.cum


#### HR Dataset PCA

In [30]:



hr = pd.read_csv("Datasets/HR_comma_sep.csv")
hr.head()
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform = 'pandas')
col_trnf = ColumnTransformer([('OHE',ohe,make_column_selector(dtype_include=object))],remainder = 'passthrough', verbose_feature_names_out=False)
col_trnf = col_trnf.set_output(transform = 'pandas')
hr1 = col_trnf.fit_transform(hr)
X = hr1.drop('left', axis = 1)
y= hr1['left']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=25, stratify=y)
scaler = StandardScaler().set_output(transform='pandas')
X_trn_scaled = scaler.fit_transform(X_train)
X_tst_scaled = scaler.transform(X_test)

pcomp = PCA().set_output(transform='pandas')
X_trn_pca = pcomp.fit_transform(X_trn_scaled)
X_trn_pca.var()
np.cumsum(pcomp.explained_variance_ratio_ *100)
lr = LogisticRegression()
lr.fit(X_trn_pca.iloc[:,:10],y_train)
X_tst_pca = pcomp.transform(X_tst_scaled)
y_pred_proba=lr.predict_proba(X_tst_pca.iloc[:,:10])
y_pred=lr.predict(X_tst_pca.iloc[:,:10])
print(accuracy_score(y_test,y_pred))
print(log_loss(y_test,y_pred_proba))


0.7815070015559014
0.46999432951219394


In [34]:
for n_features in [8, 9, 10, 11, 12, 13, 14, 15, 16]:
    X_trn_scl = scaler.fit_transform(X_train)
    X_trn_pca = prcomp.fit_transform(X_trn_scl)
    lr.fit(X_trn_pca.iloc[:, :n_features], y_train)

    X_tst_scaled = scaler.transform(X_test)
    X_tst_pca = pcomp.transform(X_tst_scaled)
    y_pred_proba=lr.predict_proba(X_tst_pca.iloc[:,:n_features])
    y_pred=lr.predict(X_tst_pca.iloc[:,:n_features])
    print("for: ", n_features)
    print(accuracy_score(y_test,y_pred))
    print(log_loss(y_test,y_pred_proba))
    print("\n")

for:  8
0.7777283840853523
0.47105835669507184


for:  9
0.78239608801956
0.47045498494563415


for:  10
0.7815070015559014
0.46999432951219394


for:  11
0.7743943098466326
0.4572792425503443


for:  12
0.7732829517670593
0.45528672950409294


for:  13
0.7732829517670593
0.45528068169510366


for:  14
0.7737274949988886
0.45465106386973336


for:  15
0.773505223382974
0.453500240575725


for:  16
0.7863969771060235
0.4352712530390712




#### making the pipeline for all the above operation and tunning the in loop for different values of components

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss, accuracy_score

for n_features in [8,9,10,11,12,13,14,15,16]:
    prcomp = PCA(n_components=n_features, svd_solver='full').set_output(transform='pandas')
    pipe = Pipeline([('SCL',scaler), ('PCA',prcomp), ('LR',lr)])
    pipe.fit(X_train, y_train)
    
    y_pred_prob = pipe.predict_proba(X_test)
    print('No. of Components:', n_features)
    print('Log Loss:', log_loss(y_test, y_pred_prob))

    y_pred = pipe.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, y_pred))

ValueError: n_components=8 must be between 0 and min(n_samples, n_features)=3 with svd_solver='full'

#### another way of making pipeline

In [ ]:
scores = []

for p_features in np.linspace(0.5, 0.95, 15):

prcomp = PCA(n_components=p_features).set_output(transform='pandas')

pipe = Pipeline([('SCL', scaler), ('PCA', prcomp), ('LR', lr)])

pipe.fit(X_train, y_train)

y_pred_prob = pipe.predict_proba(X_test)

scores.append(p_features*100,

I

log_loss(y_test, y_pred_prob), accuracy_score(y_test, y_pred))

df_scores = pd.DataFrame (scores, columns=['perc_var', 'logloss', 'acc'])